In [1]:
import tensorflow as tf

In [32]:
import numpy as np
from io import BytesIO
from PIL import Image

In [68]:
images = []
path = r"C:\Users\Cristiano Lavoro\Downloads\tpu-getting-started\tfrecords-jpeg-192x192\train\00-192x192-798.tfrec"

In [93]:
raw_dataset = tf.data.TFRecordDataset(path)

In [94]:
for item,label in raw_dataset:
    image = tf.io.decode_raw(item, tf.uint8)
    print(image.numpy().shape)
    break

TypeError: Cannot iterate over a scalar tensor.

In [77]:
feature_description = {
    'image_raw': tf.io.FixedLenFeature([], tf.string),  # For image data (stored as bytes)
}

In [78]:
def _parse_function(example_proto):
    # Parse the input tf.train.Example proto using the feature description dictionary
    return tf.io.parse_single_example(example_proto, feature_description)

# Apply the parsing function to the dataset
parsed_dataset = raw_dataset.map(_parse_function)

In [79]:
for parsed_record in parsed_dataset.take(1):
    image_raw = parsed_record['image_raw'].numpy()  # Raw bytes for the image
    
    # Do something with the features, like decoding the image
    print(f'Image: {image_raw[:20]}...')  # Preview the first few bytes

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error in user-defined function passed to MapDataset:67 transformation with iterator: Iterator::Root::Prefetch::FiniteTake::ParallelMapV2: Feature: image_raw (data type: string) is required but could not be found.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]] [Op:IteratorGetNext] name: 